# Coursera Capstone Project - Denver Restaurant Placement Analysis

### Importing & Installing Python Libraries

In [2]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
!conda install -c conda-forge folium=0.5.0 --yes
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup

print("Libraries imported.")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.6.20          |   py36h9880bd3_2         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    openssl-1.1.1h             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

### Importing List of Neighborhoods In Denver from Wikipedia Using Beautiful Soup Package

In [3]:
data = requests.get("https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Denver").text

In [4]:
soup = BeautifulSoup(data, 'html.parser')

In [5]:
nList = []

In [6]:
soup

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of neighborhoods in Denver - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"8728e948-b2df-4ca8-8dcc-ef5859c5c383","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_neighborhoods_in_Denver","wgTitle":"List of neighborhoods in Denver","wgCurRevisionId":983291351,"wgRevisionId":983291351,"wgArticleId":3721952,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Neighborhoods in Denver","Lists of neighborhoods in U.S. cities"],"wgPageContentLanguage":"en","wgPageContentModel

In [7]:
print(soup.prettify)

<bound method Tag.prettify of <!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of neighborhoods in Denver - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"8728e948-b2df-4ca8-8dcc-ef5859c5c383","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_neighborhoods_in_Denver","wgTitle":"List of neighborhoods in Denver","wgCurRevisionId":983291351,"wgRevisionId":983291351,"wgArticleId":3721952,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Neighborhoods in Denver","Lists of neighborhoods in U.S. cities"],"wgPageContentLang

### Data Cleaning

In [8]:
for row in soup.find_all("div", class_="mw-parser-output")[0].findAll("li"):
    nList.append(row.text)
    if soup.find(class_ = "mw-body-content") or soup.find(class_ = "mw-headline" or "mw-editsection-bracket") or soup.find(href = "#%"):
        continue

In [9]:
df = pd.DataFrame({"Neighborhood": nList})

In [10]:
df2 = df[13:104]
df2
dfa = df[13:20]
dfb = df[20:25]
dfc = df[25:28]
Xdfc2 = df[28:29]
dfc3 = df[29:30]
dfd = df[30:35]
Xdfe = df[35:40]
dff = df[40:45]
Xdfg = df[45:50]
Xdfh = df[50:55]
dfi = df[55:60]
Xdfj = df[60:65]
dfk = df[65:70]
dfl = df[70:75]
dfm = df[75:80]
dfn = df[80:85]
dfo = df[85:90]
Xdfp = df[90:95]
Xdfq = df[95:100]
Xdfr = df[100:104]

dffinal = pd.concat([dfa, dfb, dfc, dfc3, dfd, dff, dfi, dfk, dfl, dfm, dfn, dfo], axis = 0)

In [11]:
dffinal
dffinal.shape

(56, 1)

### Get GPS Coordinates of Denver Neighborhoods

In [12]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="example app")
geolocator.geocode("Denver, United States").raw

{'place_id': 234888713,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'relation',
 'osm_id': 253750,
 'boundingbox': ['39.6143154', '39.9142087', '-105.1098845', '-104.5996889'],
 'lat': '39.7392364',
 'lon': '-104.9848623',
 'display_name': 'Denver, Denver County, Colorado, United States of America',
 'class': 'boundary',
 'type': 'administrative',
 'importance': 0.9054758739095791,
 'icon': 'https://nominatim.openstreetmap.org/images/mapicons/poi_boundary_administrative.p.20.png'}

In [44]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get coordinates
    while(lat_lng_coords is None):
        g = geolocator.geocode('{}, Denver, United States'.format(neighborhood))
        lat_lng_coords = g
        
        
    return lat_lng_coords

In [14]:
coords = [get_latlng(neighborhood) for neighborhood in dffinal["Neighborhood"].tolist()]

In [15]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [16]:
coordinates = df_coords['Longitude']

In [17]:
df_coords.rename(columns={'Latitude':'Neighborhood','Longitude':'GPS'}, inplace = True)
dffinal=df_coords
dffinal

,Neighborhood,GPS
0,"Baker, Denver, Denver County, Colorado, United...","(39.711594500000004, -104.99374985708542)"
1,"Capitol Hill, Denver, Denver County, Colorado,...","(39.7358754, -104.9799213)"
2,"Central Business District, Denver, Denver Coun...","(39.7500931, -104.9954351)"
3,"Cherry Creek, Denver, Denver County, Colorado,...","(39.6636098, -104.8774436)"
4,"Cheesman Park, 1599, Cheesman Park, Denver, De...","(39.73281355, -104.96645529888413)"
5,"City Park, Denver, Denver County, Colorado, Un...","(39.74734, -104.95028497370055)"
6,"Congress Park, Denver, Denver County, Colorado...","(39.7337198, -104.9483668)"
7,"City Park West, Denver, Denver County, Colorad...","(39.74537615, -104.96657726439688)"
8,"Civic Center, Denver, Denver County, Colorado,...","(39.738181, -104.9877439)"
9,"Country Club, Wolters Village, Mineral Wells, ...","(32.8248337, -98.0605864)"


In [18]:
lat = [39.711594500000004,39.7358754,39.7500931,39.6636098,39.73281355,39.74734,39.7337198,39.74537615,39.738181,32.8248337,39.7331384,39.7456241,41.0544276,39.75363,39.7023138,39.7406288,39.7330213,39.78307865,37.539283749999996,45.7048887,39.731735,39.76563005,39.78295795,39.7546578,39.78073155,39.75682585,39.787909,39.76158305,39.7506207,39.7867787,39.7524265,39.6719007,39.7434647,39.66694365,41.4594827,39.7020811,39.7020811,39.66052985,31.980673,48.5402849,48.5402849,37.9308025,39.65112095,37.5688237,46.6786351,39.67466375,39.67466375,36.5189503,39.7036815,39.71768305,39.717155149999996,39.6891921,39.6835984,39.736379299999996,39.71830095,39.7307265]
lon = [-104.99374985708542,-104.9799213,-104.9954351,-104.8774436,-104.96645529888413,-104.95028497370055,-104.9483668,-104.96657726439688,-104.9877439,-98.0605864,-105.0052409729397,-104.98159787979148,-104.895525,-105.0007481,-104.9472404,-104.8977476,-104.931128,-104.99396107499159,-122.24944089661915,-95.519095,-104.9129478,-104.96655706375016,-104.95811286177516,-104.9779857,-104.9869723275022,-104.92173198175104,-105.01099347721791,-105.01250026995918,-105.01977930852223,-105.04396785379609,-105.0392401926602,-104.98049875528524,-105.0043402,-104.9384116,-87.7232711,-104.971033692151,-104.971033692151,-104.95009148581619,-102.61515,-98.6542903,-98.6542903,-121.24579786822119,-104.90715363615266,-120.1193489,-111.1737139,-105.0391671580048,-105.0391671580048,-89.6125789,-105.01074128924688,-105.03245529638046,-105.0464972167263,-105.03914145138785,-105.0072046,-105.02145887063874,-105.01582317422992,-105.0392123630247]
dffinal['Latitude'] = lat
dffinal['Longitude'] = lon

In [19]:
dffinal

,Neighborhood,GPS,Latitude,Longitude
0,"Baker, Denver, Denver County, Colorado, United...","(39.711594500000004, -104.99374985708542)",39.711595,-104.993750
1,"Capitol Hill, Denver, Denver County, Colorado,...","(39.7358754, -104.9799213)",39.735875,-104.979921
2,"Central Business District, Denver, Denver Coun...","(39.7500931, -104.9954351)",39.750093,-104.995435
3,"Cherry Creek, Denver, Denver County, Colorado,...","(39.6636098, -104.8774436)",39.663610,-104.877444
4,"Cheesman Park, 1599, Cheesman Park, Denver, De...","(39.73281355, -104.96645529888413)",39.732814,-104.966455
5,"City Park, Denver, Denver County, Colorado, Un...","(39.74734, -104.95028497370055)",39.747340,-104.950285
6,"Congress Park, Denver, Denver County, Colorado...","(39.7337198, -104.9483668)",39.733720,-104.948367
7,"City Park West, Denver, Denver County, Colorad...","(39.74537615, -104.96657726439688)",39.745376,-104.966577
8,"Civic Center, Denver, Denver County, Colorado,...","(39.738181, -104.9877439)",39.738181,-104.987744
9,"Country Club, Wolters Village, Mineral Wells, ...","(32.8248337, -98.0605864)",32.824834,-98.060586


### Map Denver Neighborhoods Using Folium

In [20]:
lat1 = 39.7392364
lon1 = -104.9848623
map_Denver = folium.Map(location=[lat1, lon1], zoom_start=10)

for lat, lon, neighborhood in zip(dffinal['Latitude'], dffinal['Longitude'], dffinal['Neighborhood']):
    
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Denver)  
    
map_Denver

### Pulling FourSquare Denver Nearby Venue Information

In [45]:
CLIENT_ID = 'HFKRWSJBAKAFUBAEMCBEM4OXJSSNETR3SGDKODB3OAK23YEN'
CLIENT_SECRET = 'BP3IWQ325YUAUSNHICGLJMMLQDWJT2HYIZ0ZWTCW4YAC3UDK'
VERSION = '20200605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HFKRWSJBAKAFUBAEMCBEM4OXJSSNETR3SGDKODB3OAK23YEN
CLIENT_SECRET:BP3IWQ325YUAUSNHICGLJMMLQDWJT2HYIZ0ZWTCW4YAC3UDK


In [22]:
radius = 700 
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat1, 
    lon1, 
    radius, 
   LIMIT)
results = requests.get(url).json()

In [23]:
venues=results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
nearby_venues.columns

Index(['reasons.count', 'reasons.items', 'referralId', 'venue.categories',
       'venue.delivery.id', 'venue.delivery.provider.icon.name',
       'venue.delivery.provider.icon.prefix',
       'venue.delivery.provider.icon.sizes', 'venue.delivery.provider.name',
       'venue.delivery.url', 'venue.events.count', 'venue.events.summary',
       'venue.id', 'venue.location.address', 'venue.location.cc',
       'venue.location.city', 'venue.location.country',
       'venue.location.crossStreet', 'venue.location.distance',
       'venue.location.formattedAddress', 'venue.location.labeledLatLngs',
       'venue.location.lat', 'venue.location.lng',
       'venue.location.neighborhood', 'venue.location.postalCode',
       'venue.location.state', 'venue.name', 'venue.photos.count',
       'venue.photos.groups', 'venue.venuePage.id'],
      dtype='object')

### Categorize Denver Venues

In [24]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
filt_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filt_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues

,name,categories,lat,lng
0,Sassafras American Eatery,Breakfast Spot,39.739949,-104.982756
1,"City, O' City",Vegetarian / Vegan Restaurant,39.736724,-104.984669
2,Kindness Yoga,Yoga Studio,39.736721,-104.984407
3,Phở-natic,Noodle House,39.740081,-104.984111
4,Civic Center Park,Park,39.739370,-104.988776
5,Shish Kabob Grill,Middle Eastern Restaurant,39.740246,-104.983633
6,Good Chemistry - Denver Dispensary,Marijuana Dispensary,39.739843,-104.982666
7,La Abeja,Mexican Restaurant,39.740026,-104.980851
8,Capitol Hill Books,Bookstore,39.739979,-104.983472
9,Denver Art Museum,Art Museum,39.736479,-104.988712


In [26]:
a=pd.Series(nearby_venues.categories)
a.value_counts()[:20]

Sandwich Place          5
Bar                     5
American Restaurant     4
Pub                     4
Burger Joint            4
Breakfast Spot          4
Pizza Place             4
Marijuana Dispensary    3
Clothing Store          2
Yoga Studio             2
Gym                     2
Dance Studio            2
Art Museum              2
Asian Restaurant        2
Gay Bar                 2
Nightclub               2
Hotel                   2
Mexican Restaurant      2
Music Venue             2
Noodle House            2
Name: categories, dtype: int64

In [46]:
def getNearbyVenues(names, latitudes, longitudes, radius=700):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        venue_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lon, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venue_results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
venueDenver = getNearbyVenues(names=dffinal['Neighborhood'],
                                   latitudes=dffinal['Latitude'],
                                   longitudes=dffinal['Longitude']
                                  )

Baker, Denver, Denver County, Colorado, United States of America
Capitol Hill, Denver, Denver County, Colorado, United States of America
Central Business District, Denver, Denver County, Colorado, 80202, United States of America
Cherry Creek, Denver, Denver County, Colorado, 80231, United States of America
Cheesman Park, 1599, Cheesman Park, Denver, Denver County, Colorado, 80206, United States of America
City Park, Denver, Denver County, Colorado, United States of America
Congress Park, Denver, Denver County, Colorado, 80206, United States of America
City Park West, Denver, Denver County, Colorado, United States of America
Civic Center, Denver, Denver County, Colorado, 80290, United States of America
Country Club, Wolters Village, Mineral Wells, Palo Pinto County, Texas, 76067, United States of America
Lincoln Park, Denver, Denver County, Colorado, United States of America
North Capitol Hill, Denver, Denver County, Colorado, United States of America
Speer, Laramie County, Wyoming, Uni

In [29]:
venueDenver.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Athmar Park, College View, Denver, Denver County, Colorado, 80223, United States of America",6,6,6,6,6,6
"Baker, Denver, Denver County, Colorado, United States of America",34,34,34,34,34,34
"Barnum West, Denver, Denver County, Colorado, United States of America",7,7,7,7,7,7
"Barnum, Denver, Denver County, Colorado, United States of America",9,9,9,9,9,9
"Bear Valley, Mariposa County, California, United States of America",1,1,1,1,1,1


### OneHot Encoding

In [30]:
DenverOH = pd.get_dummies(venueDenver[['Venue Category']], prefix="", prefix_sep="")
DenverOH['Neighborhood'] = venueDenver['Neighborhood'] 
fixed_columns = [DenverOH.columns[-1]] + list(DenverOH.columns[:-1])
DenverOH = DenverOH[fixed_columns]
Denvergroupby = DenverOH.groupby('Neighborhood').mean().reset_index()
DenverOH.head(50)

,Zoo Exhibit,ATM,Alternative Healer,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Astrologer,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Bookstore,Botanical Garden,Breakfast Spot,Brewery,Bridal Shop,Bridge,Building,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Chiropractor,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Factory,Farm,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Flea Market,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Ski Area,Ski Trail,Smoke Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Storage Facility,Strip Club,Supplement Shop,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Track,Trade School,Trail,Train Station,Travel & Transport,Tree,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio,Zoo
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Baker, Denver, Denver County, Colorado, United...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [31]:
num_top_venues = 10
for hood in Denvergroupby['Neighborhood']:
    print("---- "+hood+" ----")
    temp = Denvergroupby[Denvergroupby['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))

---- Athmar Park, College View, Denver, Denver County, Colorado, 80223, United States of America ----
                           venue  freq
0                Warehouse Store  0.33
1             Light Rail Station  0.17
2                         Bakery  0.17
3                           Park  0.17
4                   Liquor Store  0.17
5           Outdoor Supply Store  0.00
6                       Pharmacy  0.00
7                      Pet Store  0.00
8          Performing Arts Venue  0.00
9  Paper / Office Supplies Store  0.00
---- Baker, Denver, Denver County, Colorado, United States of America ----
                           venue  freq
0           Marijuana Dispensary  0.12
1             Mexican Restaurant  0.09
2                            Bar  0.06
3                 Hardware Store  0.03
4                 Shipping Store  0.03
5  Paper / Office Supplies Store  0.03
6                 Cosmetics Shop  0.03
7                Thai Restaurant  0.03
8              Convenience Store  0.03
9   

### Most Common Venues by Neighborhood

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
indic = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for indics in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(indics+1, indic[indics]))
    except:
        columns.append('{}th Most Common Venue'.format(indics+1))
nhoods_venues_sorted = pd.DataFrame(columns=columns)
nhoods_venues_sorted['Neighborhood'] = Denvergroupby['Neighborhood']
for indics in np.arange(Denvergroupby.shape[0]):
    nhoods_venues_sorted.iloc[indics, 1:] = return_most_common_venues(Denvergroupby.iloc[indics, :], num_top_venues)
    
nhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Athmar Park, College View, Denver, Denver Coun...",Warehouse Store,Bakery,Light Rail Station,Liquor Store,Park,Fast Food Restaurant,Factory,Farm,Farmers Market,Flea Market
1,"Baker, Denver, Denver County, Colorado, United...",Marijuana Dispensary,Mexican Restaurant,Bar,Coffee Shop,Beach Bar,Food Truck,Spa,Shipping Store,Chinese Restaurant,Organic Grocery
2,"Barnum West, Denver, Denver County, Colorado, ...",Mexican Restaurant,Discount Store,American Restaurant,Home Service,Convenience Store,Electronics Store,Donut Shop,Food,Farmers Market,Fast Food Restaurant
3,"Barnum, Denver, Denver County, Colorado, Unite...",Mexican Restaurant,Thrift / Vintage Store,Hookah Bar,Liquor Store,Burger Joint,Convenience Store,Marijuana Dispensary,Market,Fast Food Restaurant,Farm
4,"Bear Valley, Mariposa County, California, Unit...",Cajun / Creole Restaurant,Zoo,Department Store,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop,Food,Flea Market
5,"Capitol Hill, Denver, Denver County, Colorado,...",Pizza Place,Sandwich Place,Bar,Coffee Shop,Convenience Store,Pub,Marijuana Dispensary,Mexican Restaurant,Burger Joint,Brewery
6,"Central Business District, Denver, Denver Coun...",Hotel,American Restaurant,Cocktail Bar,Bar,New American Restaurant,Mexican Restaurant,Lounge,Restaurant,Breakfast Spot,Seafood Restaurant
7,"Chaffee Park, Denver, Denver County, Colorado,...",Grocery Store,Thrift / Vintage Store,Convenience Store,Bar,Clothing Store,Liquor Store,Mexican Restaurant,Park,Cosmetics Shop,Hobby Shop
8,"Cheesman Park, 1599, Cheesman Park, Denver, De...",Coffee Shop,Garden,Botanical Garden,Bar,Café,Park,Gift Shop,Marijuana Dispensary,Sculpture Garden,Track
9,"Cherry Creek, Denver, Denver County, Colorado,...",Park,Trail,Construction & Landscaping,Farmers Market,Financial or Legal Service,Fabric Shop,Factory,Farm,Fast Food Restaurant,Zoo


In [33]:
import matplotlib.pyplot as plt

plt.xticks(rotation = 90)
plt.bar(x=nhoods_venues_sorted['Neighborhood'], height=nhoods_venues_sorted['1st Most Common Venue'], width=0.8, bottom=True, align='center', data=nhoods_venues_sorted)

<BarContainer object of 50 artists>

In [34]:
Denver_gc = Denvergroupby.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=5, random_state=0).fit(Denver_gc)
kmeans.labels_

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 4,
       0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0], dtype=int32)

In [40]:
Denver_merged =dffinal.iloc[:16,:]

Denver_merged = Denver_merged.join(nhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
Denver_merged

,Neighborhood,GPS,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Baker, Denver, Denver County, Colorado, United...","(39.711594500000004, -104.99374985708542)",39.711595,-104.993750,0.0,Marijuana Dispensary,Mexican Restaurant,Bar,Coffee Shop,Beach Bar,Food Truck,Spa,Shipping Store,Chinese Restaurant,Organic Grocery
1,"Capitol Hill, Denver, Denver County, Colorado,...","(39.7358754, -104.9799213)",39.735875,-104.979921,0.0,Pizza Place,Sandwich Place,Bar,Coffee Shop,Convenience Store,Pub,Marijuana Dispensary,Mexican Restaurant,Burger Joint,Brewery
2,"Central Business District, Denver, Denver Coun...","(39.7500931, -104.9954351)",39.750093,-104.995435,0.0,Hotel,American Restaurant,Cocktail Bar,Bar,New American Restaurant,Mexican Restaurant,Lounge,Restaurant,Breakfast Spot,Seafood Restaurant
3,"Cherry Creek, Denver, Denver County, Colorado,...","(39.6636098, -104.8774436)",39.663610,-104.877444,0.0,Park,Trail,Construction & Landscaping,Farmers Market,Financial or Legal Service,Fabric Shop,Factory,Farm,Fast Food Restaurant,Zoo
4,"Cheesman Park, 1599, Cheesman Park, Denver, De...","(39.73281355, -104.96645529888413)",39.732814,-104.966455,0.0,Coffee Shop,Garden,Botanical Garden,Bar,Café,Park,Gift Shop,Marijuana Dispensary,Sculpture Garden,Track
5,"City Park, Denver, Denver County, Colorado, Un...","(39.74734, -104.95028497370055)",39.747340,-104.950285,0.0,Zoo Exhibit,Science Museum,Park,History Museum,Outdoor Sculpture,Track,American Restaurant,Baseball Field,General Entertainment,Gift Shop
6,"Congress Park, Denver, Denver County, Colorado...","(39.7337198, -104.9483668)",39.733720,-104.948367,0.0,Ice Cream Shop,Grocery Store,Dry Cleaner,Pizza Place,Coffee Shop,Chinese Restaurant,New American Restaurant,Burger Joint,Botanical Garden,Tapas Restaurant
7,"City Park West, Denver, Denver County, Colorad...","(39.74537615, -104.96657726439688)",39.745376,-104.966577,0.0,Mexican Restaurant,Café,Pharmacy,American Restaurant,Pizza Place,Coffee Shop,Brewery,Breakfast Spot,Liquor Store,Convenience Store
8,"Civic Center, Denver, Denver County, Colorado,...","(39.738181, -104.9877439)",39.738181,-104.987744,0.0,Hotel,Sandwich Place,Coffee Shop,Pub,Marijuana Dispensary,Spa,Burger Joint,Yoga Studio,Breakfast Spot,Bar
9,"Country Club, Wolters Village, Mineral Wells, ...","(32.8248337, -98.0605864)",32.824834,-98.060586,3.0,Wine Bar,Zoo,Event Space,Food Truck,Food Service,Food Court,Food & Drink Shop,Food,Flea Market,Financial or Legal Service


### K-Means Clustering & Map

In [47]:
kclusters = 5
# create map
map_clusters = folium.Map(location=[lat1, lon1], zoom_start=11)

# color scheme for the clusters
x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)
# add markers to the map

markers_colors = []
for lat, lon, nei , cluster in zip(Denver_merged['Latitude'], 
                                   Denver_merged['Longitude'], 
                                   Denver_merged['Neighborhood'], 
                                   Denver_merged['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow,
        fill=True,
        fill_color=rainbow,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#00b5eb', '#80ffb4', '#ffb360', '#ff0000']


### Conclusion

The analysis provided valuable information about the most common venues in Denver, sorted by the 55 neighborhoods that compose Denver. This information will allow prospective restaurant owners the ability to find optimal placement for their restaruants to make sure there is not too much direct competition in the neighborhood of choice. For example, Capitol Hill might not be the most optimal location for a pizza place because the most common restaurant type in that neighborhood is already pizza places. The competition might hinder the ability for a new pizza place to enter the market.